In [2]:
#!pip install supabase tf2onnx

In [3]:
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, GRU, Dense
from sklearn.preprocessing import MinMaxScaler
import pickle
from supabase import create_client, Client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import plotly.express as px
import tensorflow as tf
import tf2onnx

2025-06-17 20:57:45.093335: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750193865.120822    4948 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750193865.129025    4948 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

In [5]:
# FUNCION DE DESCARGA DE DATOS 
def get_demanda_data():
    end = datetime.now(timezone.utc)
    start = end - timedelta(days=3650)
    response = (
        supabase.table("demanda")
        .select("datetime,value")
        .gte("datetime", start.isoformat())
        .lte("datetime", end.isoformat())
        .execute()
    )
    df = pd.DataFrame(response.data)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.sort_values('datetime', inplace=True)
    return df

# FUNCION DE ESCALADO Y GUARDADO DEL SCALER
def escalar_datos(df, save_path='scaler.pkl'):
    scaler = MinMaxScaler()
    df['value_scaled'] = scaler.fit_transform(df[['value']])

    with open(save_path, 'wb') as f:
        pickle.dump(scaler, f)

    print(f" Datos escalados y scaler guardado en '{save_path}'.")
    return df, scaler

# FUNCION PARA DIVIDIR EN TRAIN Y TEST
def dividir_train_test(df, test_size=0.2):
    df = df.copy()
    df = df.set_index('datetime')
    split_point = int(len(df) * (1 - test_size))
    df_train = df.iloc[:split_point]
    df_test = df.iloc[split_point:]
    return df_train, df_test


In [6]:
# Cargar datos
df = get_demanda_data()
df, scaler = escalar_datos(df)
# Guardar dataset como CSV para usarlo en Streamlit
df.to_csv('datos_prediccion.csv', index=False)
print("✅ Dataset guardado como 'datos_prediccion.csv'")
df_train, df_test = dividir_train_test(df)

def crear_secuencias(datos, n_pasos):
    X, y = [], []
    for i in range(len(datos) - n_pasos):
        X.append(datos[i:i + n_pasos])
        y.append(datos[i + n_pasos])
    return np.array(X), np.array(y)

 Datos escalados y scaler guardado en 'scaler.pkl'.
✅ Dataset guardado como 'datos_prediccion.csv'


In [7]:
# Configuración de hiperparámetros
model_type = 'SimpleRNN'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mse'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 40
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

Epoch 1/40


2025-06-17 20:57:51.552716: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0334 - val_loss: 0.0104
Epoch 2/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0098 - val_loss: 0.0100
Epoch 3/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0063 - val_loss: 0.0054
Epoch 4/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0043 - val_loss: 0.0040
Epoch 5/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0040 - val_loss: 0.0032
Epoch 6/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0031 - val_loss: 0.0029
Epoch 7/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0027 - val_loss: 0.0025
Epoch 8/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0026 - val_loss: 0.0022
Epoch 9/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0021 - val_loss: 0.0022
Epoch 10/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0025 - val_loss: 0.0027
Epoch 11/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0029 - val_loss: 0.0020
Epoch 12/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0018 - val_loss: 0.002

I0000 00:00:1750193884.526644    4948 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750193884.526851    4948 single_machine.cc:361] Starting new session
I0000 00:00:1750193884.731865    4948 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750193884.732133    4948 single_machine.cc:361] Starting new session
I0000 00:00:1750193884.848726    4948 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled


✅ Modelo SimpleRNN_model_mse.onnx entrenado y guardado correctamente.
Scaler SimpleRNN guardado.
Historial de entrenamiento SimpleRNN guardado.


In [8]:
# Gráfico de pérdida
fig_loss = px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento')
fig_loss.show()

In [9]:
# Configuración de hiperparámetros
model_type = 'LSTM'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mse'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 40
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

Epoch 1/40


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - loss: 0.2206 - val_loss: 0.0421
Epoch 2/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0406 - val_loss: 0.0316
Epoch 3/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0359 - val_loss: 0.0280
Epoch 4/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0322 - val_loss: 0.0228
Epoch 5/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0240 - val_loss: 0.0161
Epoch 6/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0169 - val_loss: 0.0107
Epoch 7/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0093 - val_loss: 0.0084
Epoch 8/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0089 - val_loss: 0.0076
Epoch 9/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0073 - val_loss: 0.0066
Epoch 10/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0065 - val_loss: 0.0064
Epoch 11/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0052 - val_loss: 0.0052
Epoch 12/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0048 - val_l

I0000 00:00:1750193907.084967    4948 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750193907.085152    4948 single_machine.cc:361] Starting new session
I0000 00:00:1750193907.332289    4948 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750193907.332474    4948 single_machine.cc:361] Starting new session


✅ Modelo LSTM_model_mse.onnx entrenado y guardado correctamente.
Scaler LSTM guardado.
Historial de entrenamiento LSTM guardado.


In [10]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()

In [11]:
# Configuración de hiperparámetros
model_type = 'GRU'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mse'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 40
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

Epoch 1/40


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.1799 - val_loss: 0.0374
Epoch 2/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0298 - val_loss: 0.0207
Epoch 3/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0226 - val_loss: 0.0172
Epoch 4/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0175 - val_loss: 0.0144
Epoch 5/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0150 - val_loss: 0.0112
Epoch 6/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0120 - val_loss: 0.0086
Epoch 7/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0088 - val_loss: 0.0066
Epoch 8/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0069 - val_loss: 0.0054
Epoch 9/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0055 - val_loss: 0.0047
Epoch 10/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0049 - val_loss: 0.0043
Epoch 11/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0045 - val_loss: 0.0039
Epoch 12/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0038 - val_l

I0000 00:00:1750193930.093850    4948 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750193930.094073    4948 single_machine.cc:361] Starting new session
I0000 00:00:1750193930.343894    4948 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750193930.344106    4948 single_machine.cc:361] Starting new session


✅ Modelo GRU_model_mse.onnx entrenado y guardado correctamente.
Scaler GRU guardado.
Historial de entrenamiento GRU guardado.


In [12]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()

In [13]:
# Configuración de hiperparámetros
model_type = 'SimpleRNN'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mae'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 50
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.1088 - val_loss: 0.0525
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0500 - val_loss: 0.0375
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0361 - val_loss: 0.0332
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0315 - val_loss: 0.0316
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0416 - val_loss: 0.0368
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0329 - val_loss: 0.0274
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0263 - val_loss: 0.0272
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0261 - val_loss: 0.0275
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0267 - val_loss: 0.0268
Epoch 10/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0260 - val_loss: 0.0242
Epoch 11/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0225 - val_loss: 0.0256
Epoch 12/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0257 - val_loss: 0

I0000 00:00:1750193947.992107    4948 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750193947.992302    4948 single_machine.cc:361] Starting new session
I0000 00:00:1750193948.859095    4948 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750193948.859333    4948 single_machine.cc:361] Starting new session


✅ Modelo SimpleRNN_model_mae.onnx entrenado y guardado correctamente.
Scaler SimpleRNN guardado.
Historial de entrenamiento SimpleRNN guardado.


In [14]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()

In [15]:
# Configuración de hiperparámetros
model_type = 'LSTM'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mae'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 50
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 0.3258 - val_loss: 0.1717
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1767 - val_loss: 0.1472
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1542 - val_loss: 0.1321
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1364 - val_loss: 0.0950
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0885 - val_loss: 0.0811
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0725 - val_loss: 0.0646
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0683 - val_loss: 0.0738
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0642 - val_loss: 0.0593
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0560 - val_loss: 0.0595
Epoch 10/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0617 - val_loss: 0.0591
Epoch 11/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0465 - val_loss: 0.0482
Epoch 12/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0506 - val_l

I0000 00:00:1750193973.856590    4948 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750193973.856808    4948 single_machine.cc:361] Starting new session
I0000 00:00:1750193974.098570    4948 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750193974.098803    4948 single_machine.cc:361] Starting new session


✅ Modelo LSTM_model_mae.onnx entrenado y guardado correctamente.
Scaler LSTM guardado.
Historial de entrenamiento LSTM guardado.


In [16]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()

In [17]:
# Configuración de hiperparámetros
model_type = 'GRU'  # Cambia por 'SimpleRNN', 'LSTM' o 'GRU' para entrenar otros modelos
loss_function = 'mae'  # Puedes cambiar por 'mse' o 'mae'
n_pasos = 24
n_epochs = 50
batch_size = 32
unidades = 50

# Crear secuencias
X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Crear secuencias para Test
X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo
if model_type == 'SimpleRNN':
    rnn_layer = SimpleRNN(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'LSTM':
    rnn_layer = LSTM(unidades, activation='tanh', input_shape=(n_pasos, 1))
elif model_type == 'GRU':
    rnn_layer = GRU(unidades, activation='tanh', input_shape=(n_pasos, 1))

model = Sequential([
    rnn_layer,
    Dense(1)
])

model.compile(optimizer='adam', loss=loss_function)

# Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=n_epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),  # ⚙️ Esto es obligatorio para tener val_loss
    verbose=1
)

# Guardar el scaler
#model.save(f'{model_type}_model_{loss_function}.keras')

with open(f'scaler_{model_type}_{loss_function}.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    
# Guardar el history
with open(f"{model_type}_history_{loss_function}.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# Guardar el modelo en ONNX
# 1. Crear el input_signature
input_signature = [tf.TensorSpec([None, n_pasos, 1], tf.float32, name="input")]

# 2. Decorar la función (no obtener concrete_func)
@tf.function(input_signature=input_signature)
def model_func(x):
    return model(x)

# 3. Convertir a ONNX directamente desde la función decorada
onnx_model, _ = tf2onnx.convert.from_function(
    model_func,  # 👉 Aquí debes pasar la función decorada, no la concrete_func
    input_signature=input_signature,
    opset=13,
    output_path=f'{model_type}_model_{loss_function}.onnx'
)

print(f'✅ Modelo {model_type}_model_{loss_function}.onnx entrenado y guardado correctamente.')
print(f'Scaler {model_type} guardado.')
print(f"Historial de entrenamiento {model_type} guardado.")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - loss: 0.3497 - val_loss: 0.1515
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1416 - val_loss: 0.1124
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1164 - val_loss: 0.0951
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0988 - val_loss: 0.0784
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0769 - val_loss: 0.0650
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0615 - val_loss: 0.0539
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0521 - val_loss: 0.0500
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0470 - val_loss: 0.0446
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0469 - val_loss: 0.0427
Epoch 10/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0418 - val_loss: 0.0397
Epoch 11/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0416 - val_loss: 0.0388
Epoch 12/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0400 - val_l

I0000 00:00:1750194001.208886    4948 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750194001.209103    4948 single_machine.cc:361] Starting new session
I0000 00:00:1750194001.467454    4948 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1750194001.467690    4948 single_machine.cc:361] Starting new session


✅ Modelo GRU_model_mae.onnx entrenado y guardado correctamente.
Scaler GRU guardado.
Historial de entrenamiento GRU guardado.


In [18]:
px.line(y=history.history['loss'], title='Pérdida durante el entrenamiento').show()